<h1> Serving embeddings </h1>

This notebook illustrates:
<ol>
<li> How to create a custom embedding as part of a regression/classification model
<li> Different ways of representing categorical variables
<li> How to serve out the embedding, as well as the original model 
</ol>


In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [4]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Creating dataset

The problem is to estimate demand for bicycles at different rental stations in New York City.  The necessary data is in BigQuery:

In [20]:
query = """
#standardsql
WITH bicycle_rentals AS (
  SELECT
    COUNT(starttime) as num_trips,
    EXTRACT(DATE from starttime) as trip_date,
    MAX(EXTRACT(DAYOFWEEK from starttime)) as day_of_week,
    start_station_id
  FROM `bigquery-public-data.new_york.citibike_trips`
  GROUP BY trip_date, start_station_id
),

rainy_days AS
(
SELECT
  date,
  (MAX(prcp) > 5) AS rainy
FROM (
  SELECT
    wx.date AS date,
    IF (wx.element = 'PRCP', wx.value/10, NULL) AS prcp
  FROM
    `bigquery-public-data.ghcn_d.ghcnd_2016` AS wx
  WHERE
    wx.id = 'USW00094728'
)
GROUP BY
  date
)

SELECT
  num_trips,
  day_of_week,
  start_station_id,
  rainy
FROM bicycle_rentals AS bk
JOIN rainy_days AS wx
ON wx.date = bk.trip_date
"""
import google.datalab.bigquery as bq
df = bq.Query(query).execute().result().to_dataframe()

In [21]:
# shuffle the dataframe to make it easier to split into train/eval later
df = df.sample(frac=1.0)
df.head()

,num_trips,day_of_week,start_station_id,rainy
120701,27,7,364,False
102674,15,6,384,False
83688,262,5,388,False
31810,269,2,459,False
72086,371,4,497,False


## Feature engineering
Let's build a model to predict the number of trips that start at each station, given that we know the day of the week and whether it is a rainy day.

Inputs to the model:
* day of week (integerized, since it is 1-7)
* station id (hash buckets, since we don't know full vocabulary. The dataset has about 650 unique values. we'll use a much larger hash bucket size, but then embed it into a lower dimension)
* rainy (true/false)

Label:
* num_trips

By embedding the station id into just 2 dimensions, we will also get to learn which stations are like each other, at least in the context of rainy-day-rentals.

### Change data type

Let's change the Pandas data types to more efficient (for TensorFlow) forms.

In [24]:
df.dtypes

num_trips           int64
day_of_week         int64
start_station_id    int64
rainy                bool
dtype: object

In [28]:
import numpy as np
df = df.astype({'num_trips': np.float32, 'day_of_week': np.int32, 'start_station_id': np.int32, 'rainy': str})
df.dtypes

num_trips           float32
day_of_week           int32
start_station_id      int32
rainy                object
dtype: object

### Scale the label to make it easier to optimize.

In [35]:
df['num_trips'] = df['num_trips'] / 1000.0

In [36]:
num_train = (int) (len(df) * 0.8)
train_df = df.iloc[:num_train]
eval_df  = df.iloc[num_train:]
print("Split into {} training examples and {} evaluation examples".format(len(train_df), len(eval_df)))

Split into 104148 training examples and 26037 evaluation examples


In [37]:
train_df.head()

,num_trips,day_of_week,start_station_id,rainy
120701,0.027,7,364,False
102674,0.015,6,384,False
83688,0.262,5,388,False
31810,0.269,2,459,False
72086,0.371,4,497,False


<h2> Creating an Estimator model </h2>

Pretty minimal, but it works!

In [55]:
import tensorflow as tf
import pandas as pd

def make_input_fn(indf, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    indf,
    indf['num_trips'],
    num_epochs=num_epochs,
    shuffle=True)

def serving_input_fn():
    feature_placeholders = {
      'day_of_week': tf.placeholder(tf.int32, [None]),
      'start_station_id': tf.placeholder(tf.int32, [None]),
      'rainy': tf.placeholder(tf.string, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)
  
def train_and_evaluate(output_dir, nsteps):
  station_embed = tf.feature_column.embedding_column(
      tf.feature_column.categorical_column_with_hash_bucket('start_station_id', 5000, tf.int32), 2)
  feature_cols = [
    tf.feature_column.categorical_column_with_identity('day_of_week', num_buckets = 8),
    station_embed,
    tf.feature_column.categorical_column_with_vocabulary_list('rainy', ['false', 'true'])
  ]
  estimator = tf.estimator.LinearRegressor(
                       model_dir = output_dir,
                       feature_columns = feature_cols)
  train_spec=tf.estimator.TrainSpec(
                       input_fn = make_input_fn(train_df, None),
                       max_steps = nsteps)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec=tf.estimator.EvalSpec(
                       input_fn = make_input_fn(eval_df, 1),
                       steps = None,
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 10,  # evaluate every N seconds
                       exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
  
import shutil
OUTDIR='./model_trained'
shutil.rmtree(OUTDIR, ignore_errors=True)
train_and_evaluate(OUTDIR, 10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd520c87cd0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './model_trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorf

## Predict using the exported model

In [50]:
%writefile test.json
{"day_of_week": 3, "start_station_id": 384, "rainy": "false"}
{"day_of_week": 4, "start_station_id": 384, "rainy": "true"}

Overwriting test.json


In [56]:
%bash
EXPORTDIR=./model_trained/export/exporter/
MODELDIR=$(ls $EXPORTDIR | tail -1)
gcloud ml-engine local predict --model-dir=${EXPORTDIR}/${MODELDIR} --json-instances=./test.json

PREDICTIONS
[0.062436774373054504]
[0.08942072093486786]


  from ._conv import register_converters as _register_converters
2018-07-09 20:38:11.170549: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



## Serving out the embedding also

To serve out the embedding, we need to use a model function (a custom estimator) so that we have access to output_alternates

In [76]:
import tensorflow as tf
import pandas as pd

def make_input_fn(indf, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    indf,
    indf['num_trips'],
    num_epochs=num_epochs,
    shuffle=True)

def serving_input_fn():
    feature_placeholders = {
      'day_of_week': tf.placeholder(tf.int32, [None]),
      'start_station_id': tf.placeholder(tf.int32, [None]),
      'rainy': tf.placeholder(tf.string, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

def model_fn(features, labels, mode):
  # linear model
  station_col = tf.feature_column.categorical_column_with_hash_bucket('start_station_id', 5000, tf.int32)
  station_embed = tf.feature_column.embedding_column(station_col, 2)  # embed dimension
  embed_layer = tf.feature_column.input_layer(features, station_embed)
  
  cat_cols = [
    tf.feature_column.categorical_column_with_identity('day_of_week', num_buckets = 8),
    tf.feature_column.categorical_column_with_vocabulary_list('rainy', ['false', 'true'])
  ]
  cat_cols = [tf.feature_column.indicator_column(col) for col in cat_cols]
  other_inputs = tf.feature_column.input_layer(features, cat_cols)
  
  all_inputs = tf.concat([embed_layer, other_inputs], axis=1)
  predictions = tf.layers.dense(all_inputs, 1)  # linear model
  
  # 2. Loss function, training/eval ops
  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, predictions)
        train_op = tf.contrib.layers.optimize_loss(
            loss = loss,
            global_step = tf.train.get_global_step(),
            learning_rate = 0.01,
            optimizer = "SGD")
        eval_metric_ops = {
            "rmse": tf.metrics.root_mean_squared_error(labels, predictions)
        }
  else:
        loss = None
        train_op = None
        eval_metric_ops = None
  
  # 3. Create predictions
  predictions_dict = {
    "predicted": predictions,
    "station_embed": embed_layer
  }
    
  # 4. Create export outputs
  export_outputs = {
    "predict_export_outputs": tf.estimator.export.PredictOutput(outputs = predictions_dict)
  }

  # 5. Return EstimatorSpec
  return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = predictions_dict,
        loss = loss,
        train_op = train_op,
        eval_metric_ops = eval_metric_ops,
        export_outputs = export_outputs)

def train_and_evaluate(output_dir, nsteps):
  estimator = tf.estimator.Estimator(
                       model_fn = model_fn,
                       model_dir = output_dir)
  train_spec=tf.estimator.TrainSpec(
                       input_fn = make_input_fn(train_df, None),
                       max_steps = nsteps)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec=tf.estimator.EvalSpec(
                       input_fn = make_input_fn(eval_df, 1),
                       steps = None,
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 10,  # evaluate every N seconds
                       exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
  
import shutil
OUTDIR='./model_trained'
shutil.rmtree(OUTDIR, ignore_errors=True)
train_and_evaluate(OUTDIR, 10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd51f8580d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './model_trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorf

In [77]:
%bash
EXPORTDIR=./model_trained/export/exporter/
MODELDIR=$(ls $EXPORTDIR | tail -1)
gcloud ml-engine local predict --model-dir=${EXPORTDIR}/${MODELDIR} --json-instances=./test.json

PREDICTED               STATION_EMBED
[-0.13162994384765625]  [-0.5743858218193054, -0.2299821525812149]
[0.1520620435476303]    [-0.5743858218193054, -0.2299821525812149]


  from ._conv import register_converters as _register_converters
2018-07-10 13:21:40.623204: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License